In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
#http://www.cs.jhu.edu/~jason/papers/#eisner-2002-tnlp

## Initialization for text examples

In [2]:
Pi = np.array([
                [0.8],
                [0.2],#Starting emissions hot and cold
                 ], float)
#print("starting prob is hot", Pi[0,0])   # N X 1 Matrix

#Transition matrix
A = np.array([
                [0.6,	0.4], # H|H , C|H,
                [0.5,	0.5], # H|C , C|C,
                 ], float)


#Emissions matrix
B = np.array([
                [0.2,	0.4, .4], # 1|H, 2|H, 3|H    #One two or three icecreams given its hot
                [0.5,	0.4, .1], # 1|C, 2|C, 3|C
                 ], float)

B = np.array([# P(...|C) P(...|H)
                [0.5,	0.2], # P(1|...)    #One two or three icecreams given its hot
                [0.4,	0.4], # P(2|...)
                [0.1,	0.4], # P(3|...)
                 ], float)


Odic = {"1":0,"2":1,"3":2}    # O is observations, 1 icecream, 2 icecream, 3 icecream
Qdic = {"H":0,"C":1}    # Q is state hot or cold
Qdicrev = {0:"H",1:"C"}    # Q is state hot or cold


#Likelihood of observation
#P(O|Q)


S = "233232322313311121"
# S= "313"

T = len(S)
num_states = len(Qdic)
num_obs = len(Odic)


## Forward with text example

In [3]:

def forwardTrellis_text(S):
    forward_trellis = np.zeros(shape=(len(Qdic), len(S)), dtype="float")

    #Initialize starting probability
    for state in range(num_states):
        P_start = Pi[state]
        initial_observation = Odic[S[0]] # Amount of icecreams to start. In this case 3. Turns it to 2 and gets prob from matrix
        state_observation_likelihood = B[initial_observation,state]
        forward_trellis[state,0] = P_start * state_observation_likelihood
                
    #Implement forward algorithm
    for n in range(T-1):
        n = n + 1 #Skip first column
        print("n is", "*"*10,n)
        for trellis_state in range(len(Qdic)):
            print("-"*15)
            print("Trellis state is", trellis_state)
            cell_sum = 0
            obs =  Odic[S[n]]
            state_observation_likelihood = B[ obs, trellis_state]
            print("state_observation_likelihood :", state_observation_likelihood)
            for switching_state in range(len(Qdic)):      #Loop through previous cell and all possible transitions in this state
                prev_forward_path_probability = forward_trellis[switching_state,n-1]
                transition_probability = A[switching_state,trellis_state]
                print("transition prob :", transition_probability)
                print("prev_forward_path_probability :", prev_forward_path_probability)
                cell_sum = cell_sum + prev_forward_path_probability * state_observation_likelihood * transition_probability
            forward_trellis[trellis_state,n] = cell_sum
        print(forward_trellis)
forwardTrellis_text(S)

n is ********** 1
---------------
Trellis state is 0
state_observation_likelihood : 0.1
transition prob : 0.6
prev_forward_path_probability : 0.32
transition prob : 0.5
prev_forward_path_probability : 0.08
---------------
Trellis state is 1
state_observation_likelihood : 0.4
transition prob : 0.4
prev_forward_path_probability : 0.32
transition prob : 0.5
prev_forward_path_probability : 0.08
[[ 0.32    0.0232  0.      0.      0.      0.      0.      0.      0.      0.
   0.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.08    0.0672  0.      0.      0.      0.      0.      0.      0.      0.
   0.      0.      0.      0.      0.      0.      0.      0.    ]]
n is ********** 2
---------------
Trellis state is 0
state_observation_likelihood : 0.1
transition prob : 0.6
prev_forward_path_probability : 0.0232
transition prob : 0.5
prev_forward_path_probability : 0.0672
---------------
Trellis state is 1
state_observation_likelihood : 0.4
transition prob : 0.4
prev_forward_

## Viterbi


In [4]:
def viterbi(S):
    V = np.zeros(shape=(lenQ, len(S)), dtype="float")
    pointers = np.zeros(shape=(lenQ, len(S)), dtype="int")
    #Viterbi
    for j in range(lenQ):
        P_start = Pi[j]
        initial_observation = Odic[S[0]] # Amount of icecreams to start. In this case 3. Turns it to 2 and gets prob from matrix
        state_observation_likelihood = B[j,initial_observation]
        V[j,0] = P_start * state_observation_likelihood

    for n in range(len(S)-1):
        n = n + 1 #Skip first column
        print("n is", "*"*10,n)
        for trellis_state in range(lenQ):
            print("-"*15)
            print("Trellis state is", trellis_state)
            M = np.zeros(shape=(1,lenQ), dtype="float")
            state_observation_likelihood = B[trellis_state, Odic[S[n]] ]
            print("state_observation_likelihood :", state_observation_likelihood)
            for switching_state in range(lenQ):      #Loop through previous cell and all possible transitions in this state
                prev_forward_path_probability = V[switching_state,n-1]
                rolling_A = np.roll(A,switching_state,axis=0)  #Loop through all transitions
                transition_probability = A[switching_state,trellis_state]
                print("transition prob :", transition_probability)
                print("prev_forward_path_probability :", prev_forward_path_probability)
                print("state likelihood :", prev_forward_path_probability * state_observation_likelihood * transition_probability)
                M[0,switching_state] = prev_forward_path_probability * state_observation_likelihood * transition_probability

            best_state = M.argmax()
            pointers[trellis_state,n] = best_state
            print("best previous state :", best_state)


            V[trellis_state,n] = M[0,best_state]
    print(V)
    print(pointers)

    print("Get best sequence")
    best_states = []
    best_end_state = V[:,len(S)-1].argmax(axis=0) #Go to last row for traceback
    best_end_states = pointers[best_end_state,:]
    for n in range(len(best_end_states)):
        decoded = Qdicrev[ best_end_states[n] ]
        best_states.append(decoded)


    best_states.reverse()
    print(best_states)

viterbi(S)

NameError: name 'lenQ' is not defined

## Reinitialization for excel examples

In [47]:

####################################################
#This function follows the excel example

Pi = np.array([
                [0.5],
                [0.5],#Starting emissions hot and cold
                 ], float)
#print("starting prob is hot", Pi[0,0])   # N X 1 Matrix

B = np.array([# P(...|C) P(...|H)
                [0.6,	0.1], # P(1|...)    #One two or three icecreams given its hot
                [0.35,	0.2], # P(2|...)
                [0.05,	0.7], # P(3|...)
                 ], float)

A = np.array([# P(...|C) P(...|H)
                [0.7,	0.09], # P(C|...)
                [0.29,	0.9], # P(H|...)
                # [0.1,	0.1], # P(STOP|...)
                 ], float)

STOP = np.array([ [0.01,	0.01],] # P(STOP|...)
               )

Odic = {"1":0,"2":1,"3":2}    # O is observations, 1 icecream, 2 icecream, 3 icecream
Qdic = {"C":0,"H":1}    # Q is state hot or cold
Qdicrev = {0:"C",1:"H"}    # Q is state hot or cold

S="322"
S="233232322313311121113121112332322"
num_states = len(Qdic)
num_obs = len(Odic)



## Forward algorithm for excel examples

In [29]:

def forwardTrellis(S,A,B,Pi):
    T = len(S)
    forward_trellis = np.zeros(shape=(T, num_states), dtype="float")

    #Initialize starting probability
    for j in range(num_states):
        P_start = Pi[j]
        initial_observation = Odic[S[0]] # Amount of icecreams to start. In this case 3. Turns it to 2 and gets prob from matrix
        state_observation_likelihood = B[initial_observation,j]
        forward_trellis[0, j] = P_start * state_observation_likelihood

    #Implement forward algorithm
    for t in range(T-1):
        t = t + 1 #Skip first column
#         print("n is", "*"*10,n)
        for j in range(num_states):
        # j is current state
#             print("-"*15)
#             print("Trellis state is", trellis_state)
            cell_sum = 0
            observation = Odic[S[t]]                   
            state_observation_likelihood = B[observation, j]
#             print("state_observation_likelihood :", state_observation_likelihood)
            for i in range(num_states):      #Loop through previous cell and all possible transitions in this state
                prev_forward_path_probability = forward_trellis[t-1, i]
                transition_probability = A[j,i]
#                 print("transition prob :", transition_probability)
#                 print("prev_forward_path_probability :", prev_forward_path_probability)
                cell_sum = cell_sum + prev_forward_path_probability * state_observation_likelihood * transition_probability
            forward_trellis[t, j] = cell_sum

    return forward_trellis

fw = forwardTrellis(S,A,B,Pi)
fw

array([[  1.75000000e-01,   1.00000000e-01],
       [  6.57500000e-03,   9.85250000e-02],
       [  6.73487500e-04,   6.34054750e-02],
       [  2.16227690e-03,   1.14520478e-02],
       [  1.27213906e-04,   7.65373231e-03],
       [  2.72259975e-04,   1.38505022e-03],
       [  1.57618251e-05,   9.27850415e-04],
       [  3.30889352e-05,   1.67927261e-04],
       [  1.33964978e-05,   3.21460651e-05],
       [  6.13534717e-07,   2.29715101e-05],
       [  1.49814613e-06,   2.08522842e-06],
       [  6.18186423e-08,   1.61781757e-06],
       [  9.44383153e-09,   1.03177425e-06],
       [  5.96822188e-08,   9.31335537e-08],
       [  3.00957438e-08,   1.01128042e-08],
       [  1.31863038e-08,   1.78292895e-09],
       [  3.28680670e-09,   1.08573283e-09],
       [  1.43908839e-09,   1.93033349e-10],
       [  6.14840923e-10,   5.91065646e-11],
       [  2.61424942e-10,   2.31499776e-11],
       [  9.25404787e-12,   6.76537491e-11],
       [  7.54000256e-12,   6.35720481e-12],
       [  

## Backward algorithm excel example


In [7]:
## TODO Finish this

def backwardTrellis(S,A,B,STOP):
    T = len(S)
    bw = np.zeros(shape=(T, num_states), dtype="float")
    
    #Initialize ending probability
    #bw[:,T-1] = [0.1,0.2] Fill in without loop
    for state in range(num_states):
        bw[T-1, state] = STOP[0,state]

#     print("T is ", T)
    for t in range(T-2,-1,-1):
        future_obs = Odic[S[t+1]] # Use future observation

        for i in range(num_states):
#             print("-"*15)
#             print("Trellis state is", trellis_state)
            cell_sum = 0
            for j in range(num_states):
#                 transition_probability = A[j,i]
            
                transition_probability = A[j, i]
                state_observation_likelihood = B[future_obs,j]
                future_back_probability = bw[ t+1, j]
#                 print("state_observation_likelihood :", state_observation_likelihood)
#                 print("transition prob :", transition_probability)
#                 print("prev_back_probability :", future_back_probability)
                cell_sum = cell_sum + future_back_probability * state_observation_likelihood * transition_probability
                
#                 prev_forward_path_probability = forward_trellis[switching_state,n-1]
#                 transition_probability = A[switching_state,trellis_state]
            bw[t, i] = cell_sum

#         print(bw)
    return bw
backwardTrellis(S,A,B,STOP)    

array([[  1.38956227e-18,   4.06898051e-18],
       [  2.33819161e-18,   6.44199785e-18],
       [  7.82225776e-18,   1.01695201e-17],
       [  1.93545343e-17,   5.31102906e-17],
       [  6.68023378e-17,   8.38248890e-17],
       [  1.69436218e-16,   4.36042490e-16],
       [  8.62403518e-16,   6.85970911e-16],
       [  2.73097060e-15,   3.33302965e-15],
       [  7.05554347e-15,   1.72821113e-14],
       [  4.43178293e-14,   2.71153667e-14],
       [  8.83771871e-14,   2.48255540e-13],
       [  2.49887786e-13,   3.92271501e-13],
       [  3.68075085e-12,   5.96362098e-13],
       [  8.66515085e-12,   1.42715502e-12],
       [  2.03807496e-11,   3.62882825e-12],
       [  4.77184811e-11,   1.16892252e-11],
       [  1.87149044e-10,   3.21890572e-11],
       [  4.39088461e-10,   9.42031146e-11],
       [  1.01523628e-09,   4.37559508e-10],
       [  2.17149746e-09,   3.55887384e-09],
       [  3.05439412e-08,   5.43083508e-09],
       [  7.15201327e-08,   1.74305324e-08],
       [  

## Forward Backward Baum Welch

#### Get Z (Total prob of Start-STOP paths)

In [8]:
def get_Z(S, alpha,beta):
    T = len(S)
    total_prob_per_state = np.zeros(shape=(T, num_states), dtype="float") 
    for t in range(T):
        for j in range(num_states):
            total_prob_per_state[t,j] = alpha[t,j] * beta[t,j]
    
    
    Z_vector = np.zeros(shape=(T, 1), dtype="float") 
    for t in range(T): # A(state1)*Bstate(1) + A(state2)*Bstate(2)
        Z = 0
        for j in range(num_states):
            Z = Z + total_prob_per_state[t,j]        
        Z_vector[t,0] = Z
        
    Z = Z_vector[0,0]
    return Z, total_prob_per_state           
    
alpha = forwardTrellis(S,A,B,Pi)
beta = backwardTrellis(S,A,B,STOP) 
Z, total_prob_per_state = get_Z(S,alpha,beta)


#### Sum prob you reached a state for ALL of ice cream data (...|C) or (...|H)

In [9]:
def get_prob_reach_state_for_all(S, Z,total_prob_per_state):
    T = len(S)
    prob_reach_state = np.zeros(shape=(T, num_states), dtype="float") 
    
    for t in range(T):
        for j in range(num_states):
            prob_reach_state[t,j] = total_prob_per_state[t,j] / Z
    
    prob_reach_state_sum = np.sum(prob_reach_state, axis=0)  
    return prob_reach_state_sum, prob_reach_state

prob_reach_state_sum, prob_reach_state = get_prob_reach_state_for_all(S, Z,total_prob_per_state)
prob_reach_state_sum


array([ 13.78823754,  19.21176246])

### Update transition matrix

#### Get Xi 

In [10]:
def get_Xi_sum(S, alpha,beta,Z):
    T = len(S)
    # Update transition matrix
    # Even though the text says we use t+1. In excel it is calculated as past alpha
    
    Xi_sum = np.zeros(shape=(num_states, num_states), dtype="float") 
    for j in range(num_states):
        Xi = np.zeros(shape=(T, num_states), dtype="float") 
        for t in range(1, T):
            curr_obs = Odic[S[t]] # Use future observation
            state_observation_likelihood = B[curr_obs,j]
            curr_beta = beta[t,j]
            
#             print("state_observation_likelihood :", state_observation_likelihood)  
            for i in range(num_states):
                prev_alpha = alpha[t-1,i]  # past alpha based on what state you came from
                transition_probability = A[j,i]
                not_quite_Xi_per_state = prev_alpha * transition_probability * state_observation_likelihood * curr_beta
                Xi[t,i] = not_quite_Xi_per_state / Z
        
                # Sum down each row of transitions
                Etsum = np.sum(Xi, axis=0)  
                # Make a new transition matrix. Now you have to divide by total transitions
                Xi_sum[j,i] = Etsum[i]    
    return Xi_sum  
        
                
Xi_sum = get_Xi_sum(S, alpha,beta, Z)

#### Update transition matrix

In [11]:
def update_transition(Xi_sum, prob_reach_state_sum):  
       
    new_A = np.zeros(shape=(num_states, num_states), dtype="float") 
    for i in range(num_states):
        for j in range(num_states):
            new_A[j,i] = Xi_sum[j,i]  / prob_reach_state_sum[i]    
    return new_A
new_A = update_transition(Xi_sum,prob_reach_state_sum)    


#### Update start and stop matrix

In [12]:

def update_stop(prob_reach_state_sum, prob_reach_state):  
    new_STOP = np.zeros(shape=(1, num_states), dtype="float") 
    for j in range(num_states):
        last_prob = prob_reach_state[-1,j]
        new_STOP[0,j] = last_prob / prob_reach_state_sum[j]    
    return new_STOP
new_STOP = update_stop(prob_reach_state_sum, prob_reach_state)

def update_Pi(prob_reach_state):  
    
    new_Pi = np.zeros(shape=(num_states,1), dtype="float") 
    for j in range(num_states):
        new_Pi[j,0] = prob_reach_state[0,j]
    return new_Pi

new_Pi = update_Pi(prob_reach_state)

### Update emission matrix

#### Sum  prob of observations    (Probability we reached a state at the end of a day  and ate 1 or 2 of 3 ice creams)

In [13]:
def get_emis_sum(S, prob_reach_state):  
    T = len(S)
    emis_sum = np.zeros(shape=(num_obs,num_states), dtype="float")
    for i in range(num_states):
        emis_per_state = np.zeros(shape=(T, num_obs), dtype="float") 
        for t in range(T):
            obs = Odic[S[t]] 
            emis_per_state[t,obs] = prob_reach_state[t,i]     
        emis_sum[:,i] = np.sum(emis_per_state, axis=0)      
    return emis_sum

emis_sum = get_emis_sum(S, prob_reach_state)

def update_B(emis_sum,prob_reach_state_sum):
    new_B = np.zeros(shape=(num_obs,num_states), dtype="float")
    for i in range(num_states):
        for obs in range(num_obs):
            new_B[obs,i] = emis_sum[obs,i] / prob_reach_state_sum[i]            
    return new_B            

new_B = update_B(emis_sum,prob_reach_state_sum)

### Complete Baum Welch

In [51]:
def Baum_Welch(S,A,B,Pi,STOP):
    T = len(S)
    # Get forward and backward probabilities
    alpha = forwardTrellis(S=S,A=A,B=B,Pi=Pi)
    beta = backwardTrellis(S=S,A=A,B=B,STOP=STOP)
    
    #Sum up total probabilities. 
        # Z                    = A(state1)*Bstate(1) + A(state2)*Bstate(2)      Number
        # total_prob_per_state = A(state1)*Bstate(1) or A(state2)*Bstate(2)     Vector
    Z, total_prob_per_state = get_Z(S ,alpha,beta)
    
    # Get vector of reaching a state per time point. The sum is just the sum of these
    # i.e. for one state and time point. Alpha(state0) * beta(state0) / Z
    prob_reach_state_sum, prob_reach_state = get_prob_reach_state_for_all(S, Z,total_prob_per_state)
    
    
    # Update transition matrix
        # sum of prev_alpha * transition_probability * state_observation_likelihood * curr_beta / Z
    Xi_sum = get_Xi_sum(S, alpha,beta, Z)
        # Get Xi sum and divide by the sum of of all probabilities of reaching that state
    A = update_transition(Xi_sum,prob_reach_state_sum)
    

    Pi = update_Pi(prob_reach_state) # Take the first time point from the probability that you reached that state
    
    # Get last final prob you reached that state, and divide by the sum of all. Do for each state
    STOP = update_stop(prob_reach_state_sum, prob_reach_state)      # last_prob / prob_reach_state_sum   
    
    # Update emission matrix
    
    #Get sum for each timepoint of prob you reached that state and obs of today. p(obs|state)
    # for each state make a [T, num_obs] matrix. If that obs happened fill in prob you reached that state. sum down time points
    emis_sum = get_emis_sum(S, prob_reach_state)
    
    # Divide your emis sum by the total prob you reached that state.
    B = update_B(emis_sum,prob_reach_state_sum)
    
    return A,B,Pi,STOP
    
new_A,new_B,new_Pi,new_STOP = Baum_Welch(S,A,B,Pi,STOP) 

array([[ 0.78497725,  0.13485035],
       [ 0.19370449,  0.82839827]])

In [81]:

#### Difference says 0. Start again here

# Test Baum Welch
A_TRUTH = A
B_TRUTH = B
Pi_TRUTH = Pi
STOP_TRUTH = STOP


Pi_init = np.array([
                [0.1],
                [0.1],#Starting emissions hot and cold
                 ], float)



STOP_init = np.array([ [0.1,	0.1],] # P(STOP|...)
               )

B_init = np.array([# P(...|C) P(...|H)
                [0.4,	0.7], # P(1|...)    #One two or three icecreams given its hot
                [0.65,	0.2], # P(2|...)
                [0.05,	0.1], # P(3|...)
                 ], float)

A_init = np.array([# P(...|C) P(...|H)
                [0.3,	0.7], # P(C|...)
                [0.6,	0.1], # P(H|...)
                # [0.1,	0.1], # P(STOP|...)
                 ], float)





def Run_Baum_Difference(S,A_TRUTH, B_TRUTH, A_init,B_init,Pi_init,STOP_init):
    A_init,B_init,Pi_init,STOP_init = Baum_Welch(S,A_init,B_init,Pi_init,STOP_init) 
    print("new B is\n",new_B)
    print("new A is\n",new_A)
    print("new Pi is\n",new_Pi)
    print("new STOP is\n",new_STOP) 
    A_diff =  np.sum( np.absolute(A_TRUTH - A) )
    B_diff =  np.sum( np.absolute(B_TRUTH - B) ) #.__sub__(B) #B_TRUTH - B
    print("A estimated is\n", A)
    print("A diff is\n", A_diff)
    print("B estimated is\n", B)
    print("B diff is\n", B_diff)
    return A_diff, B_diff



def Graph_Baum_Iterations(S,A_TRUTH, B_TRUTH, A_init,B_init,Pi_init,STOP_init, iterations):
    feature_list = ["Difference", "Iterations","Matrix_type"]
    df = pd.DataFrame(0, index=np.arange(iterations), columns=feature_list)
    

    for x in range(0,iterations):
        print("*********"*10)
        print("x is", x)
        A_diff, B_diff = Run_Baum_Difference(S,A_TRUTH, B_TRUTH, A_init,B_init,Pi_init,STOP_init)
        df["Iterations"].iloc[x] = x
        df["Difference"].iloc[x] = A_diff
        df["Matrix_type"].iloc[x] ="Transition"
    
    
#     ax = sns.pointplot(x="Iterations", y="Difference", hue="Matrix_type",data=df)
    ax = sns.pointplot(x="Iterations", y="Difference",data=df)
    for item in ax.get_xticklabels():
        item.set_rotation(90)

    fig = ax.get_figure()
    
    print(df)
    
    fig.savefig('GRAPHS/BAUM_ITERATIONS:'+ str(iterations) +".png")
        
        
Graph_Baum_Iterations(S,A_TRUTH, B_TRUTH, A_init,B_init,Pi_init,STOP_init, iterations=4)

#     df["Iterations"].iloc[x] = len(S)
# #     df["length"].iloc[x+runs] = HMM.length
#     df["Difference"].iloc[x] = A_diff
# #     df["Difference"].iloc[x+runs] = B_diff
#     df["Matrix_type"].iloc[x] ="Transition"
#     df["Matrix_type"].iloc[x+runs] = "Observation"

# print(df)

# df = df.sort_values(by="length")
# ax = sns.pointplot(x="iterations", y="Difference", hue="Matrix_type",
#                     data=df)
# for item in ax.get_xticklabels():
#     item.set_rotation(90)

# fig = ax.get_figure()
# fig.savefig('GRAPHS/Matrix_differences, iterations:'+ str(iterations) +" runs:"+ str(runs) + ".png")


******************************************************************************************
x is 0
new B is
 [[ 0.67834077  0.08572229]
 [ 0.27149503  0.37771454]
 [ 0.0501642   0.53656316]]
new A is
 [[ 0.78497725  0.13485035]
 [ 0.19370449  0.82839827]]
new Pi is
 [[ 0.3740718]
 [ 0.6259282]]
new STOP is
 [[ 0.02131826  0.03675138]]
A estimated is
 [[ 0.7   0.09]
 [ 0.29  0.9 ]]
A diff is
 0.0
B estimated is
 [[ 0.6   0.1 ]
 [ 0.35  0.2 ]
 [ 0.05  0.7 ]]
B diff is
 0.0
******************************************************************************************
x is 1
new B is
 [[ 0.67834077  0.08572229]
 [ 0.27149503  0.37771454]
 [ 0.0501642   0.53656316]]
new A is
 [[ 0.78497725  0.13485035]
 [ 0.19370449  0.82839827]]
new Pi is
 [[ 0.3740718]
 [ 0.6259282]]
new STOP is
 [[ 0.02131826  0.03675138]]
A estimated is
 [[ 0.7   0.09]
 [ 0.29  0.9 ]]
A diff is
 0.0
B estimated is
 [[ 0.6   0.1 ]
 [ 0.35  0.2 ]
 [ 0.05  0.7 ]]
B diff is
 0.0
**************************************************

/Users/M/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


******************************************************************************************
x is 3
new B is
 [[ 0.67834077  0.08572229]
 [ 0.27149503  0.37771454]
 [ 0.0501642   0.53656316]]
new A is
 [[ 0.78497725  0.13485035]
 [ 0.19370449  0.82839827]]
new Pi is
 [[ 0.3740718]
 [ 0.6259282]]
new STOP is
 [[ 0.02131826  0.03675138]]
A estimated is
 [[ 0.7   0.09]
 [ 0.29  0.9 ]]
A diff is
 0.0
B estimated is
 [[ 0.6   0.1 ]
 [ 0.35  0.2 ]
 [ 0.05  0.7 ]]
B diff is
 0.0
   Difference  Iterations Matrix_type
0         0.0           0  Transition
1         0.0           1  Transition
2         0.0           2  Transition
3         0.0           3  Transition


/Users/M/anaconda3/lib/python3.5/site-packages/seaborn/categorical.py:1460: FutureWarning: remove_na is deprecated and is a private function. Do not use.
  stat_data = remove_na(group_data)


## Benchmarking

#### Create sequence generator

In [43]:

from numpy.random import choice
class HMMGEN(object):
    # The class "constructor" - It's actually an initializer
    def __init__(self, A, B, Pi, STOP):
        self.hiddenstate = 0
        self.emission = 0
        self.S = []
        self.O = []
        self.GENERATE = True
        self.x = 0
        self.R = 0.0
        self.A = A
        self.B = B
        self.Pi = Pi
        self.STOP = STOP
        self.num_obs = B.shape[0]
        self.num_obs_list = [state for state in range(self.num_obs)]
        self.num_states = B.shape[1]
        self.num_states_list = [state for state in range(self.num_states)]
        ASUM = np.sum(self.A, axis=0)
        ONES = np.ones(shape=(1, num_states), dtype="float")
        STOP = ONES - ASUM
        self.ASTOP = np.concatenate([A, STOP], axis=0) # Concat Transition and STOP matrix
        
    def StartProb(self):       
        probabilities = Pi[:,0]     # Transform Pi matrix to a list
        self.hiddenstate = choice(self.num_states_list, p=probabilities)
        self.S.append(str(self.hiddenstate))

    def makeEmission(self):
        probabilities = B[:,self.hiddenstate]   # Get observations for that hidden state
        current_emmision = choice(self.num_obs_list, p=probabilities)
        self.O.append(str(current_emmision+1))         #Add one to the because icecream starts at one

    def makeTransition(self):
        self.num_states_list_with_stop = [state for state in range(self.num_states+1)]
        
        # Do 1 - transition probabilities to get stopping probabilities
        probabilities = self.ASTOP[:,self.hiddenstate]   # Get observations for that hidden state
        
        self.hiddenstate = choice(self.num_states_list_with_stop, p=probabilities)
        self.S.append(str(self.hiddenstate))

        if self.hiddenstate == num_states: # Stop state
            self.GENERATE = False
            
    def Generate(self):
        while self.GENERATE == True:
            self.makeEmission()
            self.makeTransition()
def makeHMMSeq(A, B, Pi, STOP):
    HMM = HMMGEN(A, B, Pi, STOP)
    HMM.StartProb()
    HMM.Generate()
    HMM.length = len(HMM.O)
    HMM.S = "".join(HMM.S)[:-1] # Take off stop state
    HMM.O = "".join(HMM.O)
    print("transition list  is, ",HMM.S)
    print("observation list is, ",HMM.O)
    print("Length of generated sequence is", HMM.length)
    return HMM

HMM = makeHMMSeq(A, B, Pi, STOP)
A_new,B_new,Pi_new,STOP_new = Baum_Welch(S=HMM.O,A=A,B=B,Pi=Pi,STOP=STOP)

transition list  is,  0
observation list is,  1
Length of generated sequence is 1


### Error Function


##### Initialization

In [40]:
A_TRUTH = A
B_TRUTH = B
Pi_TRUTH = Pi
STOP_TRUTH = STOP

Pi_init = np.array([
                [0.1],
                [0.1],#Starting emissions hot and cold
                 ], float)



STOP_init = np.array([ [0.1,	0.1],] # P(STOP|...)
               )

B_init = np.array([# P(...|C) P(...|H)
                [0.4,	0.7], # P(1|...)    #One two or three icecreams given its hot
                [0.65,	0.2], # P(2|...)
                [0.05,	0.1], # P(3|...)
                 ], float)

A_init = np.array([# P(...|C) P(...|H)
                [0.3,	0.7], # P(C|...)
                [0.6,	0.1], # P(H|...)
                # [0.1,	0.1], # P(STOP|...)
                 ], float)

print("A start is \n", A_init)
print("A TRUTH is\n", A_TRUTH)
print("B TRUTH is\n", B_TRUTH)
print("B start is \n", B_init)

A start is 
 [[ 0.3  0.7]
 [ 0.6  0.1]]
A TRUTH is
 [[ 0.7   0.09]
 [ 0.29  0.9 ]]
B TRUTH is
 [[ 0.6   0.1 ]
 [ 0.35  0.2 ]
 [ 0.05  0.7 ]]
B start is 
 [[ 0.4   0.7 ]
 [ 0.65  0.2 ]
 [ 0.05  0.1 ]]


##### Run the random sequence

In [41]:





####################################
#Run it and graph the difference
def runRandomSequence(S, A,B, Pi, STOP, iterations):
        # Give starting parameters for B and A
        x = 0
        while x < iterations:
            A,B,Pi,STOP = Baum_Welch(S=S,A=A,B=B,Pi=Pi,STOP=STOP)
            x= x + 1
        return A,B,Pi,STOP
def getDifference(A,B,A_TRUTH,B_TRUTH):
        B_diff =  np.sum( np.absolute(B_TRUTH - B) ) #.__sub__(B) #B_TRUTH - B
        A_diff =  np.sum( np.absolute(A_TRUTH - A) )
        return A_diff, B_diff
HMM = makeHMMSeq(A, B, Pi, STOP)  
iterations=5
A_new,B_new,Pi_new,STOP_new = runRandomSequence(S=HMM.O, A=A_init, B=B_init, Pi=Pi_init, STOP=STOP_init, iterations=iterations)

transition list  is,  010011000011111111111111111111111111110011111011111100111111110000001111111011111001111111100111111111111111111101111111111110000111111111111000011111
observation list is,  331232211133333333123321323131333123232233323111113211213333331111213312332133233113311333312333313333331332323312331333323331211233232322332121133333
Length of generated sequence is 150


In [42]:

x= 0
feature_list = ["length", "Difference","Matrix_type"]
runs = 5
iterations = 20
df = pd.DataFrame(0, index=np.arange(runs*2), columns=feature_list)

for x in range(0,runs):
    HMM = makeHMMSeq(A_TRUTH, B_TRUTH, Pi_TRUTH, STOP_TRUTH)
    A,B,Pi,STOP = runRandomSequence(S=HMM.O, A=A_init, B=B_init, Pi=Pi_init, STOP=STOP_init, iterations=iterations)
    A_diff, B_diff = getDifference(A,B,A_TRUTH,B_TRUTH)
    df["length"].iloc[x] = HMM.length
    df["length"].iloc[x+runs] = HMM.length
    df["Difference"].iloc[x] = A_diff
    df["Difference"].iloc[x+runs] = B_diff
    df["Matrix_type"].iloc[x] ="Transition"
    df["Matrix_type"].iloc[x+runs] = "Observation"
    print("A estimated is\n", A)
    print("A diff is\n", A_diff)
    print("B estimated is\n", B)
    print("B diff is\n", B_diff)


df = df.sort_values(by="length")
ax = sns.pointplot(x="length", y="Difference", hue="Matrix_type",
                    data=df)
for item in ax.get_xticklabels():
    item.set_rotation(90)

fig = ax.get_figure()
fig.savefig('GRAPHS/Matrix_differences, iterations:'+ str(iterations) +" runs:"+ str(runs) + ".png")



transition list  is,  0000000001111000000000000111111111111111111111111111111101111111111111
observation list is,  2212122223333222211112111231223211323333333112233233231323333331332323
Length of generated sequence is 70
A estimated is
 [[ 0.15116543  0.00601778]
 [ 0.1344625   0.17725587]]
A diff is
 1.51109841507
B estimated is
 [[ 0.47761193  0.08395966]
 [ 0.51282212  0.24416261]
 [ 0.00956595  0.67187773]]
B diff is
 0.413969460414
transition list  is,  011111111111111111001111111111111111111100000000000000011111111111111111111111000000111111
observation list is,  132223221332333333223231333133211133333222111212122122223333313222312332332331211222333331
Length of generated sequence is 90
A estimated is
 [[ 0.16693896  0.00316278]
 [ 0.25190867  0.17835303]]
A diff is
 1.37963656619
B estimated is
 [[ 0.42077596  0.11207604]
 [ 0.55733444  0.24388598]
 [ 0.02188961  0.64403798]]
B diff is
 0.526592916439
transition list  is,  00111
observation list is,  11113
Length of generated se

/Users/M/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


A estimated is
 [[ 0.1916366   0.00823685]
 [ 0.05786716  0.21913475]]
A diff is
 1.50312463815
B estimated is
 [[ 1.          0.43433603]
 [ 0.          0.        ]
 [ 0.          0.56566397]]
B diff is
 1.46867205838
transition list  is,  01110000000000111100111111111111111111
observation list is,  11311111111111131111333313133131333333
Length of generated sequence is 38
A estimated is
 [[  2.60539600e-01   3.78524232e-04]
 [  4.47738324e-01   2.56054531e-01]]
A diff is
 1.33076566856
B estimated is
 [[ 0.89757566  0.25156838]
 [ 0.          0.        ]
 [ 0.10242434  0.74843162]]
B diff is
 1.1
transition list  is,  01111100000000000011110000000111111111111111111111110011111111110011111111100001110011111100111111111101111111111111111111111111110000
observation list is,  13133311113111111131331131111313313313311333331313331133333331331133311313311111331113333113333333333113333331113333331133133311331111
Length of generated sequence is 134
A estimated is
 [[  8.16425519e-02   3.438092

/Users/M/anaconda3/lib/python3.5/site-packages/seaborn/categorical.py:1508: FutureWarning: remove_na is deprecated and is a private function. Do not use.
  stat_data = remove_na(group_data[hue_mask])
